# bytes, str, unicode의 차이점을 알자

|Python3|Python2|
|:-|:-|
|bytes와 str 두 가지 타입으로 문자 시퀀스 표현|str과 unicode 두 가지 타입으로 문자 시퀀스 표현|
|bytes instance는 raw 8bit 값 저장|str instance는 raw 8bit 값 저장|
|str instance는 unicode 문자 저장|unicode instance는 unicode 문자 저장|

## Binary data(raw 8bit) 표현 방법

- unicode 문자를 binary data(raw 8bit)로 표현하는 방법은 많음 (대표적으로 UTF-8)
- Python3 str instance와 Python2 unicode instance는 binary data로 변환하기 위해 encode method 사용
- binary data를 unicode 문자로 변환하기 위해 decode method 사용
- Python programming 시 외부에 제공할 interface에서 unicode를 encode, decode 해야 함
- Program 핵심 부분에서 unicode type을 사용하고, 문자 encoding에 대해서는 어떤 가정도 하지 말아야 함

출력 텍스트 인코딩을 엄격하게 유지하면서 다른 텍스트 인코딩을 쉽게 수용 가능

## 문자 타입의 분리로 인한 두가지 상황

1. 인코드된 문자인 raw 8bit 값을 처리하려는 상황
2. 인코딩이 없는 문자를 처리하려는 상황

원하는 타입과 입력값의 타입이 정확히 일치하려면 핼퍼 함수 두개가 필요

## Python3

### str이나 bytes를 입력으로 받고 str을 반환하는 method

In [11]:
def to_str(bytes_or_str):
    if isinstance(bytes_or_str, bytes):
        value = bytes_or_str.decode('utf-8')
    else:
        value = bytes_or_str
    return value # str instance

### str이나 bytes를 받고 bytes를 반환하는 method

In [12]:
def to_bytes(bytes_or_str):
    if isinstance(bytes_or_str, str):
        value = bytes_or_str.encode('utf-8')
    else:
        value = bytes_or_str
    return value # bytes instance

## Python2

### str이나 unicode를 입력으로 받고 unicode를 반환하는 method

In [13]:
def to_unicode(unicode_or_str):
    if isinstance(unicode_or_str, str):
        value = unicode_or_str.decode('utf-8')
    else:
        value = unicode_or_str
    return value # unicode instance

### str이나 unicode를 받고 str을 반환하는 method

In [14]:
def to_str(unicode_or_str):
    if isinstance(unicode_or_str, unicode):
        value = unicode_or_str.encode('utf-8')
    else:
        value = unicode_or_str
    return value # str instance

## Python에서 raw 8bit 값과 unicode 문자를 처리할 때 이슈

### 1. Python2에서 str이 7bit ASCII 문자만 포함한다면 unicode와 str instance가 같은 타입처럼 보임

- str과 unicode를 '+' 연산자로 묶을 수 있음
- 같음(equality)과 같지 않음(inequality) 연산자로 이런 str과 unicode를 비교 가능
- '%s' 같은 포맷 문자열에 unicode instance 사용 가능

7bit ASII 만 처리하는 경우 str 또는 unicode를 받는 함수에 str이나 unicode instance를 넘겨도 문제없이 동작함을 의미  
Python3 에서는 bytes와 str instance는 빈 문자열이라도 절대 같지 않으므로, 함수에 넘기는 문자열의 타입을 더 신주히 처리

### 2. Python3에서 내장 함수 open이 반환하는 파일 핸들을 사용하는 연산은 기본으로 UTF-8 인코딩 사용

Python2의 경우 binary encoding이 기본이다.  
예를 들어 임의의 binary data를 파일에 기록하려 할때, 아래 코드는 Python2에서는 동작하지만 Python3에서는 동작하지 않는다.

In [15]:
import os

with open('/tmp/random.bin','w') as f:
    f.write(os.urandom(10))

TypeError: write() argument must be str, not bytes

문제의 이유는 Python3의 open에 새로운 encoding 인수가 추가되었기 때문이다. __파라미터의 기본 값은 'utf-8'이기 때문에 파일 핸들을 사용하는 read, write연산은 binary data를 담은 bytes instance가 아닌 unicode 문자를 담은 str instance를 기대한다.__ 위 코드를 동작하게 하려면 data를 문자 쓰기 모드('w')가 아니라 __binary 쓰기 모드('wb')로 오픈__ 해야 한다.

Python2와 Python3에서 open이 올바르게 동작하게 만드는 방법은 다음과 같다.

In [16]:
import os

with open('/tmp/random.bin','wb') as f:
    f.write(os.urandom(10))

이 문제는 파일에서 데이터를 읽어올 때 발생한다. binary data를 파일에서 읽거나 쓸때 binary mode('rb' or 'wb')로 오픈한다.